In [1]:
from attention import AttentionLayer

import re

import numpy as np
import pandas as pd 

from pymystem3 import Mystem
from nltk.corpus import stopwords
from tensorflow.keras.models import load_model
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

c:\users\020gudaevii\desktop\dev\venvs\tex_gen\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\020gudaevii\desktop\dev\venvs\tex_gen\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\020gudaevii\desktop\dev\venvs\tex_gen\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\020gudaevii\desktop\dev\venvs\t

In [2]:
models_dir = "../models/"

encoder = "encoder_v1.h5"
decoder = "decoder_v1.h5"

path_to_tokenizer = "../data/tokenizer.pickle"
path_to_vocab = "../data/vacob.json"
test_comments = "../data/raw_data/test_comments.json"

In [3]:
russian_stopwords = stopwords.words("russian")
mystem = Mystem()

In [4]:
import json

with open(path_to_vocab,'r', encoding='utf8') as f:
    data = json.load(f)
    

In [5]:
import pickle

with open(path_to_tokenizer,'rb') as handle:
    tokenizer = pickle.load(handle)

In [6]:
def preprocess_text(text, lemmatize=False):

    #удаление разрывов строк
    text = text.replace('\r', ' ')
    text = text.replace('\n', ' ')

    #удаление хештегов и отметок
    tokens = re.sub(r'[#]\w+','', text.lower())
    tokens = re.sub(r'[@]\w+','', tokens)
    
    if lemmatize:
        #удаление всех символов кроме букв и пробелов
        tokens = re.sub(r'[^а-я\s]', '', tokens)
        tokens = mystem.lemmatize(tokens)
        tokens = [token for token in tokens if token not in russian_stopwords
                        and token != " " and token != "\n"]
    
        tokens = " ".join(tokens)
    else:
        #удаление всех символов кроме букв, пробелов и знаков препинания
        tokens = re.sub(r'[^а-я\s.,:;!?\-1]', '', tokens)

    text = re.sub(r'(\s)\1+', r'\1', tokens)

    text = text.replace('\n', '')
    #if len(text) > 1:
    return text

In [7]:
reverse_target_word_index = data['reverse_target_word_index']
reverse_source_word_index = data['reverse_source_word_index']
target_word_index = data['target_word_index']
max_summary_len = data['max_summary_len']
max_text_len = data['max_text_len']

In [8]:
encoder_model = load_model(models_dir + encoder, compile=True)
decoder_model = load_model(models_dir + decoder, custom_objects={'AttentionLayer': AttentionLayer}, compile=True)

W0902 12:00:20.199356  7128 deprecation.py:506] From c:\users\020gudaevii\desktop\dev\venvs\tex_gen\lib\site-packages\tensorflow\python\keras\initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0902 12:00:20.201356  7128 deprecation.py:506] From c:\users\020gudaevii\desktop\dev\venvs\tex_gen\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0902 12:00:20.201856  7128 deprecation.py:506] From c:\users\020gudaevii\desktop\dev\venvs\tex_gen\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceS

In [9]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[str(sampled_token_index)]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [10]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [44]:
def predict(description):
    
    bold = "\033[1m"
    reset = "\033[0;0m"
    
    print(bold,"Оригинальный текст:",reset, description)
    print('-' * 100)
    
    text = preprocess_text(description, True)
    
    print(bold, "Лемматизированный текст:", reset, text)
    print('-' * 100)
    
    example = tokenizer.texts_to_sequences([text])

    example = pad_sequences(example, maxlen=max_text_len, padding='post')

    gen = decode_sequence(example.reshape(1,max_text_len))
    
    print(bold, "Коммент сети:", reset, gen)
    print('=' * 100)

In [45]:
with open(test_comments,'r',encoding='utf8') as f:
    comments = json.load(f)
    
for i in comments:
    predict(i)

 Оригинальный текст:  В Грозном прошло заседание дискуссионной площадки национального проекта «Демография».
Совещание с участием депутатов Парламента ЧР, регионального совета Сторонников "Единой России". Общественнной Палаты ЧР, Совета Профсоюзов прошло в Чеченском государственном педагогическом университете.
Главные цели национального проекта «Демография» - увеличение ожидаемой продолжительности здоровой жизни до 67 лет, снижение смертности населения старше трудоспособного возраста, увеличение суммарной рождаемости.
Также одними из основных целей являются увеличение доли граждан, ведущих здоровый образ жизни, а также увеличение до 55 процентов доли граждан, систематически занимающихся физической культурой и спортом.
Общий объем финансирования пяти региональных проектов нацпроекта на 2019-2024гг. составляет почти 8 млрд.руб. #ЧГТРК #ЧЕЧНЯ #ГРОЗНЫЙ #НАЦПРОКТ
----------------------------------------------------------------------------------------------------
 Лемматизированный текст:  гр

 Лемматизированный текст:  чечня проверять исламский магазин поле зрение специалист роспотребнадзор совет экономический общественный безопасность биологически активный добавка речь называть препарат пчелка это добавка понимать прочитывать простой обыватель удаваться упаковка отсутствовать маркировка описание экспертиза также выявлять наличие капсула предназначать лечение сустав сразу несколько вид психотропный вещество проверка проводить поручение глава республика слово надзорный орган продолжать 
----------------------------------------------------------------------------------------------------
 Коммент сети:   а где можно посмотреть
 Оригинальный текст:  ОБРАЩЕНИЕ К РАМЗАНУ КАДЫРОВУ.

Чеченцы из Иордании благодарят Рамзана Кадырова за все позитивные преобразования в Чечне.
⠀
Председатель правления общественного благотворительного фонда «Друзья чечено-ингушской республики», руководитель Совета чеченских тейпов в Иордании Самих Бено отметил, что родные и близкие молодых ребят, которые

 Лемматизированный текст:  село махкета веденский район открываться школа место свой страница социальный сеть вк сообщать глава чеченский республика рамзан кадыров строительство школа финансировать региональный общественный фонд имя герой россия ахматхадж кадыров присваивать имя уходить воля аллах праведный мир руководитель управление ростехнадзор чеченский республика шайхи хазуева шайхи газимагомедович истинно верующий мусульманин порядочный уважаемый общество человек настоящий патриот наш отечество написать новый образовательный учреждение представлять трхэтажный здание который создавать весь условие организация качественный учебный процесс награждать медаль заслуга чеченский республика житель садиться майрбек абдулазизов заниматься реализация инвестиционный проект ставить задача близкий будущее создавать чечня тысяча ученический место конец текущий год чечня построить целый ряд новый школа уверенный это позволять обеспечивать каждый ребнка комфортный условие учба ликвидировать трхсм